In [ ]:
import pickle
import os
import numpy as np
import pandas as pd
from scipy import integrate

path = '/nobackup/jakravit/data/HE60/output/EcoLight/excel/'
dirlist = os.listdir(path)
outFile = path + dirlist[0]
print(dirlist[0])

In [ ]:
kparheadlist = ['depth', 'K_PAR (from Eo)', 'K_d (broadband)']
parheadlist = ['depth', 'PAR_Eo', 'PAR_Ed']
bheadlist = ['scat coef b']
aheadlist = ['abs coef a']

line_num = 0
with open(outFile, 'r') as f:
    lines = f.readlines()

for line in lines:
    line_num += 1
    if all(x in line.strip().replace('"', '').replace("'", "") for x in kparheadlist):
        kparheader = line_num
        l1 = line

    elif all(x in line.strip().replace('"', '').replace("'", "") for x in parheadlist):
        parheader = line_num 
        l2 = line

    elif all(x in line.strip().replace('"', '').replace("'", "") for x in bheadlist):
        bheader = line_num 
        l3 = line

    elif all(x in line.strip().replace('"', '').replace("'", "") for x in aheadlist):
        aheader = line_num
        l4 = line

    else:
        pass  

In [ ]:
# lines = np.arange(header,header+201,1)
# end = header+201

kparline = kparheader
parline = parheader + 1
blines = np.arange(bheader+2, bheader+203, 1)
alines = np.arange(aheader+2, aheader+203, 1)
end = aheader+203

kpar = []
par = []
b = []
a = []
wl = []

i = 0
with open(outFile, "r+") as fp:
    # access each line
    while True:
        line = fp.readline()
        
        # check line number
        if i == kparline:
            kpar.append(line.strip().split('   ')[1])
        elif i == parline:
            par.append(line.strip().split('   ')[1])
        elif i in blines:
            b.append(line.strip().split('   ')[1] )
        elif i in alines:
            a.append(line.strip().split('   ')[1] )
        
        if i > end:
            break;
        i = i + 1

kpar = float(kpar[0])
par = float(par[0])
a = np.array(a).astype(float)
b = np.array(b).astype(float)
c = a + b
ctot685 = c[114] # 685 nm 
ctot740 = c[136] # 740 nm 


In [ ]:
uid = 'mvil2cizg6'
bname = '/nobackup/jakravit/data/HE60/batch_data/case1V3_inputs.csv'
bdata = pd.read_csv(bname,index_col=0)
aphyEuk = bdata.filter(regex='^aphyEuk')
aphyCy = bdata.filter(regex='^aphyCy')
fqy = bdata.loc[uid,'FQY']
# qa = bdata.loc[uid,'Qa']
qa = np.random.choice([.3,.4,.5,.6])

In [ ]:
aphyEukSim = aphyEuk.loc[uid,:]
aphyEukPar = aphyEukSim[:121].values

aphyCySim = aphyCy.loc[uid,:]
aphyCyPar = aphyCySim[:121].values

l = np.arange(400, 702.5, 2.5)
ps = np.random.choice(np.arange(.1,.21, .01))
absdphy = integrate.simps(aphyEukPar, l) * par + \
        ((integrate.simps(aphyCyPar, l) * ps) * par)

ktot_g1 = kpar + ctot685
ktot_g2 = kpar + ctot740
    
absd_g1 = absdphy / ktot_g1
absd_g2 = absdphy / ktot_g2

lf_g1 = (.54 * (1 / (4 * np.pi)) * (fqy/25) * qa) * absd_g1
lf_g2 = (.54 * (1 / (4 * np.pi)) * (fqy/50) * aa) * absd_g2

flamp_g1 = (lf_g1 / 1.23) * .001
flamp_g2 = (lf_g2 / 1.23) * .001